In [ ]:
# Initialize your connection to Jinko

import sys; sys.path.insert(0, '../lib')
import jinko_helpers as jinko

# Configuration

# Fill your API key (ex: 'd74cc07e-4a86-4aab-952a-a5814ae6e1ee')
# If not set, it will be retrieved from the JINKO_API_KEY environment variable.
# If environment variable is not set, you will be asked for it interactively
apiKey = None

# Fill your Project Id (ex: '14495063-918c-441a-a898-3131d70b02b0')
# If not set, it will be retrieved from the JINKO_PROJECT_ID environment variable.
# If environment variable is not set, you will be asked for it interactively
projectId = None

# This function ensures that authentication is correct
# It it also possible to override the base url by passing baseUrl=...
jinko.initialize(projectId, apiKey = apiKey)

In [ ]:
# Listing available packages
#help("modules")

In [ ]:
import zipfile
import io
from typing import List, Any

# Fill the short Id of your Trial (ex: tr-EKRx-3HRt)
trialId = 'tr-OxkW-mB8I'
if trialId is None:
  raise Exception('Please specify a Trial Id')

# Convert short Id to coreItemId
coreItemId = jinko.getCoreItemId(trialId, 1)

# List all Trial versions
versions: List[Any] = jinko.makeRequest('/core/v2/trial_manager/trial/%s/status'%(coreItemId['id'])).json()

# Get the latest completed version
latestCompletedVersion = next((item for item in versions if item['status'] == 'completed'), None)
if latestCompletedVersion is None:
  raise Exception('No completed Trial version found')

# Retrieve time series
response = jinko.makeRequest('/core/v2/result_manager/timeseries_summary', method='POST'
    , json={
      "select": ["Blood.Drug"],
      "trialId": latestCompletedVersion['simulationId']
    })
archive = zipfile.ZipFile(io.BytesIO(response.content))
filename = archive.namelist()[0]
print(filename)

csvTimeseries = archive.read(filename).decode('utf-8')

# Retrieve scalar results
response = jinko.makeRequest('/core/v2/result_manager/scalars_summary', method='POST', json={
  "select": [
    "fullDose.tmin",
    "primingDose.tmin",
    "administrationMode"
  ],
  "trialId": latestCompletedVersion['simulationId']
})
archive = zipfile.ZipFile(io.BytesIO(response.content))
filename = archive.namelist()[0]
print(filename)

csvScalars = archive.read(filename).decode('utf-8')


In [ ]:
import pandas as pd

# Load timeseries into a dataframe
dfTs = pd.read_csv(io.StringIO(csvTimeseries))

print("Timeseries data : \n")
print(dfTs[1:3])
print("\n")

# Load scalars into a dataframe
dfScalars = pd.read_csv(io.StringIO(csvScalars))

# Pivot to a wide format 
dfScalars = dfScalars.pivot(index = ["armId","patientId"], columns="scalarId", values = "value")

print("Pivotted scalar table: \n")
print(dfScalars[1:3])
print("\n")

# Merge both tables together to obtain protocol descriptors per arm
df = dfTs.merge(right=dfScalars,left_on=["Patient Id","Arm"],right_on=["patientId","armId"])

print("Merged table: \n")
print(df[1:3])
print("\n")

# Convert time to days
df["Time"]=df["Time"].map(lambda x: x / (60 * 60 * 24))

In [ ]:
import plotly.express as px
import plotly.io as pio
# This config was required in order for the plot to show in the notebook
pio.renderers.default = 'iframe'

fig = px.line(df,
              x="Time",
              y="Value",
              facet_col="fullDose.tmin", 
              facet_row="primingDose.tmin",
              color="administrationMode",
              labels={
                "Time": "Time (days)",
                "Value": "Concentration (µg/mL)",
                "fullDose.tmin": "Full dose (mg)",
                "primingDose.tmin": "Priming dose (mg)",
                "administrationMode": "Administration"
                },
              log_y=True
             )
fig.show()